In [1]:
#importing numpy and pandas which are required for data pre-processing
import numpy as np
import pandas as pd

In [2]:
#Loading the data
raw_data = pd.read_csv('100-legitimate-art.txt')

In [3]:
msg = 'http://www.emuck.com:3000/archive/egan.html'

In [4]:
import pandas as pd

In [5]:
data = {'websites':[msg]}
raw_data = pd.DataFrame(data)

In [6]:
raw_data.head()

,websites
0,http://www.emuck.com:3000/archive/egan.html


In [7]:
raw_data['websites'].str.split("://").head() 

0    [http, www.emuck.com:3000/archive/egan.html]
Name: websites, dtype: object

In [8]:
seperation_of_protocol = raw_data['websites'].str.split("://",expand = True)

In [9]:
seperation_of_protocol.head()

,0,1
0,http,www.emuck.com:3000/archive/egan.html


In [10]:
type(seperation_of_protocol)

pandas.core.frame.DataFrame

In [11]:
seperation_domain_name = seperation_of_protocol[1].str.split("/",1,expand = True)

In [12]:
seperation_domain_name.head()

,0,1
0,www.emuck.com:3000,archive/egan.html


In [13]:
type(seperation_domain_name)

pandas.core.frame.DataFrame

In [14]:
seperation_domain_name.columns=["domain_name","address"]

In [15]:
seperation_domain_name.head()

,domain_name,address
0,www.emuck.com:3000,archive/egan.html


In [16]:
#Concatenation of data frames
splitted_data = pd.concat([seperation_of_protocol[0],seperation_domain_name],axis=1)

In [17]:
splitted_data.columns = ['protocol','domain_name','address']

In [18]:
splitted_data.head()

,protocol,domain_name,address
0,http,www.emuck.com:3000,archive/egan.html


In [19]:
type(splitted_data)

pandas.core.frame.DataFrame

In [20]:
def long_url(l):
    """This function is defined in order to differntiate website based on the length of the URL"""
    if len(l) < 54:
        return 0
    elif len(l) >= 54 and len(l) <= 75:
        return 2
    return 1

In [21]:
#Applying the above defined function in order to divide the websites into 3 categories
splitted_data['long_url'] = raw_data['websites'].apply(long_url) 

In [22]:
#Will show the results only the websites which are legitimate according to above condition as 0 is legitimate website
splitted_data[splitted_data.long_url == 0].head()

,protocol,domain_name,address,long_url
0,http,www.emuck.com:3000,archive/egan.html,0


In [23]:
def have_at_symbol(l):
    """This function is used to check whether the URL contains @ symbol or not"""
    if "@" in l:
        return 1
    return 0

In [24]:
splitted_data['having_@_symbol'] = raw_data['websites'].apply(have_at_symbol)

In [25]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol
0,http,www.emuck.com:3000,archive/egan.html,0,0


In [26]:
def redirection(l):
    """If the url has symbol(//) after protocol then such URL is to be classified as phishing """
    if "//" in l:
        return 1
    return 0

In [27]:
splitted_data['redirection_//_symbol'] = seperation_of_protocol[1].apply(redirection)

In [28]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol
0,http,www.emuck.com:3000,archive/egan.html,0,0,0


In [29]:
def prefix_suffix_seperation(l):
    if '-' in l:
        return 1
    return 0

In [30]:
splitted_data['prefix_suffix_seperation'] = seperation_domain_name['domain_name'].apply(prefix_suffix_seperation)

In [31]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0


In [32]:
def sub_domains(l):
    if l.count('.') < 3:
        return 0
    elif l.count('.') == 3:
        return 2
    return 1

In [33]:
splitted_data['sub_domains'] = splitted_data['domain_name'].apply(sub_domains)

In [34]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0,0


In [35]:
import re
def having_ip_address(url):
    match=re.search('(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  #IPv4
                    '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)'  #IPv4 in hexadecimal
                    '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}',url)     #Ipv6
    if match:
        #print match.group()
        return 1
    else:
        #print 'No matching pattern found'
        return 0

In [36]:
splitted_data['having_ip_address'] = raw_data['websites'].apply(having_ip_address)

In [37]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0,0,0


In [38]:
#we have imported re module in the above feature. So need not to import again
def shortening_service(url):
    match=re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                    'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                    'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                    'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                    'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                    'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                    'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|tr\.im|link\.zip\.net',url)
    if match:
        return 1
    else:
        return 0

In [39]:
splitted_data['shortening_service'] = raw_data['websites'].apply(shortening_service)

In [40]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address,shortening_service
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0,0,0,0


In [41]:
def https_token(url):
    match=re.search('https://|http://',url) 
    if match.start(0)==0:
        url=url[match.end(0):]
    match=re.search('http|https',url)
    if match:
        return 1
    else:
        return 0

In [42]:
splitted_data['https_token'] = raw_data['websites'].apply(https_token)

In [43]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address,shortening_service,https_token
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0,0,0,0,0


In [44]:
def abnormal_url_sub(domain,url):
    hostname=domain.name
    match=re.search(hostname,url)
    if match:
        return 0
    else:
        return 1

In [45]:
def abnormal_url_main(domain):
    dns = 0
    try:
        domain_name = whois.whois(domain)
    except:
        dns = 1
        
    if dns == 1:
        return 1
    else:
        return abnormal_url_sub(domain_name)

In [46]:
splitted_data['abnormal_url'] = splitted_data['domain_name'].apply(abnormal_url_main)

In [47]:
splitted_data

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address,shortening_service,https_token,abnormal_url
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0,0,0,0,0,1


In [50]:
X = splitted_data[['long_url','having_@_symbol','redirection_//_symbol','prefix_suffix_seperation','sub_domains','having_ip_address','shortening_service','https_token','abnormal_url']].values

In [54]:
X[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int64)

In [55]:
from statistics import mode
y = mode(X[0])

In [56]:
y

0

In [48]:
import google
def google_index(url):
    site=google.search(url,5)
    if site:
        return 0
    else:
        return 1

In [49]:
splitted_data['google_index'] = raw_data['websites'].apply(google_index)

AttributeError: module 'google' has no attribute 'search'

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
def web_traffic(url):
    try:
        rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find("REACH")['RANK']
    except TypeError:
        return 1
    rank= int(rank)
    if (rank<100000):
        return 0
    else:
        return 2

In [ ]:
splitted_data['web_traffic'] = raw_data['websites'].apply(web_traffic)

In [ ]:
splitted_data.head()

In [ ]:
!pip install whois

In [ ]:
import whois
from datetime import datetime
import time
def domain_registration_length_sub(domain):
    expiration_date = domain.expiration_date
    today = time.strftime('%Y-%m-%d')
    today = datetime.strptime(today, '%Y-%m-%d')
    if expiration_date is None:
        return 1
    elif type(expiration_date) is list or type(today) is list :
        return 2             #If it is a type of list then we can't select a single value from list. So,it is regarded as suspected website  
    else:
        registration_length = abs((expiration_date - today).days)
        if registration_length / 365 <= 1:
            return 1
        else:
            return 0

In [ ]:
def domain_registration_length_main(domain):
    dns = 0
    try:
        domain_name = whois.whois(domain)
    except:
        dns = 1
        
    if dns == 1:
        return 1
    else:
        return domain_registration_length_sub(domain_name)

In [ ]:
splitted_data['domain_registration_length'] = splitted_data['domain_name'].apply(domain_registration_length_main)

In [ ]:
splitted_data.head()

In [ ]:
#Testing the above function
domain_registration_length_main('www.google.com')

In [ ]:
domain_registration_length_main('www.w3schools.com')

In [ ]:
def age_of_domain_sub(domain):
    creation_date = domain.creation_date
    expiration_date = domain.expiration_date
    if ((expiration_date is None) or (creation_date is None)):
        return 1
    elif ((type(expiration_date) is list) or (type(creation_date) is list)):
        return 2
    else:
        ageofdomain = abs((expiration_date - creation_date).days)
        if ((ageofdomain/30) < 6):
            return 1
        else:
            return 0

In [ ]:
def age_of_domain_main(domain):
    dns = 0
    try:
        domain_name = whois.whois(domain)
    except:
        dns = 1
        
    if dns == 1:
        return 1
    else:
        return age_of_domain_sub(domain_name)

In [ ]:
splitted_data['age_of_domain'] = splitted_data['domain_name'].apply(age_of_domain_main)

In [ ]:
splitted_data

In [ ]:
def dns_record(domain):
    dns = 0
    try:
        domain_name = whois.whois(domain)
        print(domain_name)
    except:
        dns = 1
        
    if dns == 1:
        return 1
    else:
        return dns

In [ ]:
splitted_data['dns_record'] = splitted_data['domain_name'].apply(dns_record)

In [ ]:
splitted_data

In [ ]:
import socket
def statistical_report(url):
    hostname = url
    h = [(x.start(0), x.end(0)) for x in re.finditer('https://|http://|www.|https://www.|http://www.', hostname)]
    z = int(len(h))
    if z != 0:
        y = h[0][1]
        hostname = hostname[y:]
        h = [(x.start(0), x.end(0)) for x in re.finditer('/', hostname)]
        z = int(len(h))
        if z != 0:
            hostname = hostname[:h[0][0]]
    url_match=re.search('at\.ua|usa\.cc|baltazarpresentes\.com\.br|pe\.hu|esy\.es|hol\.es|sweddy\.com|myjino\.ru|96\.lt|ow\.ly',url)
    try:
        ip_address = socket.gethostbyname(hostname)
        ip_match=re.search('146\.112\.61\.108|213\.174\.157\.151|121\.50\.168\.88|192\.185\.217\.116|78\.46\.211\.158|181\.174\.165\.13|46\.242\.145\.103|121\.50\.168\.40|83\.125\.22\.219|46\.242\.145\.98|107\.151\.148\.44|107\.151\.148\.107|64\.70\.19\.203|199\.184\.144\.27|107\.151\.148\.108|107\.151\.148\.109|119\.28\.52\.61|54\.83\.43\.69|52\.69\.166\.231|216\.58\.192\.225|118\.184\.25\.86|67\.208\.74\.71|23\.253\.126\.58|104\.239\.157\.210|175\.126\.123\.219|141\.8\.224\.221|10\.10\.10\.10|43\.229\.108\.32|103\.232\.215\.140|69\.172\.201\.153|216\.218\.185\.162|54\.225\.104\.146|103\.243\.24\.98|199\.59\.243\.120|31\.170\.160\.61|213\.19\.128\.77|62\.113\.226\.131|208\.100\.26\.234|195\.16\.127\.102|195\.16\.127\.157|34\.196\.13\.28|103\.224\.212\.222|172\.217\.4\.225|54\.72\.9\.51|192\.64\.147\.141|198\.200\.56\.183|23\.253\.164\.103|52\.48\.191\.26|52\.214\.197\.72|87\.98\.255\.18|209\.99\.17\.27|216\.38\.62\.18|104\.130\.124\.96|47\.89\.58\.141|78\.46\.211\.158|54\.86\.225\.156|54\.82\.156\.19|37\.157\.192\.102|204\.11\.56\.48|110\.34\.231\.42',ip_address)  
    except:
        return 1

    if url_match:
        return 1
    else:
        return 0

In [ ]:
splitted_data['statistical_report'] = raw_data['websites'].apply(statistical_report)

In [ ]:
splitted_data

In [ ]:
whois